In [1]:
import numpy as np
import os 
os.environ['TF_ENABLE_ONEDNN_OPTS']='0'
import tensorflow as tf 
import tensorflow_datasets as tfds
from tensorflow import keras
import matplotlib.pyplot as plt

In [2]:
ALL_DATA_DIR = '/media/windows/'
training = ALL_DATA_DIR + 'training/'
validation = ALL_DATA_DIR + 'validation/'
scaled = ALL_DATA_DIR + 'scaled/'
scaled_wo_rotate = ALL_DATA_DIR + "scaled_wo_rotate/"

scaled_wo_rotate_training = scaled_wo_rotate + "training/"
scaled_wo_rotate_validation = scaled_wo_rotate + "validation/"

datFile = training + "shape_predictor_68_face_landmarks.dat"

In [3]:
batch_size = 32 
img_height = 224
img_width = 224 

In [4]:
validation_s = tf.keras.utils.image_dataset_from_directory(scaled_wo_rotate_validation, labels = 'inferred',
            image_size=(img_height,img_width), batch_size=batch_size, seed=42).prefetch(1)

Found 50947 files belonging to 7 classes.


2022-09-02 00:27:23.982548: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-02 00:27:23.991574: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-02 00:27:23.994252: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-02 00:27:23.997414: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the approp

In [5]:
training_s = tf.keras.utils.image_dataset_from_directory(scaled_wo_rotate_training, labels = 'inferred',
            image_size=(img_height,img_width), batch_size=batch_size, seed=42).prefetch(1)

Found 399839 files belonging to 7 classes.


In [6]:
base_model = keras.applications.MobileNetV3Small(input_shape=(224,224,3), include_top = False, weights=None)

In [7]:
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(7, activation='softmax')(avg)
model = keras.Model(inputs=base_model.input, outputs=output)

In [8]:
opt = keras.optimizers.Adam(learning_rate=1e-4)

In [9]:
es = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

In [10]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
history = model.fit(training_s, validation_data=validation_s, epochs=40, callbacks=[es])
model.save('0902_mobile_no_weight.h5')

Epoch 1/40


2022-09-02 00:28:02.634470: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8400


12495/12495 [==============================] - 816s 65ms/step - loss: 1.1742 - accuracy: 0.5494 - val_loss: 0.9840 - val_accuracy: 0.6322
Epoch 2/40
12495/12495 [==============================] - 802s 64ms/step - loss: 0.8793 - accuracy: 0.6708 - val_loss: 0.8471 - val_accuracy: 0.6875
Epoch 3/40
12495/12495 [==============================] - 805s 64ms/step - loss: 0.7812 - accuracy: 0.7098 - val_loss: 0.7938 - val_accuracy: 0.7080
Epoch 4/40
12495/12495 [==============================] - 809s 65ms/step - loss: 0.7109 - accuracy: 0.7368 - val_loss: 0.7758 - val_accuracy: 0.7196
Epoch 5/40
12495/12495 [==============================] - 812s 65ms/step - loss: 0.6555 - accuracy: 0.7582 - val_loss: 0.7757 - val_accuracy: 0.7240
Epoch 6/40
12495/12495 [==============================] - 805s 64ms/step - loss: 0.6083 - accuracy: 0.7760 - val_loss: 0.8185 - val_accuracy: 0.7123
Epoch 7/40
12495/12495 [==============================] - 802s 64ms/step - loss: 0.5679 - accuracy: 0.7916 - val_loss

In [11]:
model.evaluate(training_s)

12495/12495 [==============================] - 370s 30ms/step - loss: 0.6056 - accuracy: 0.7772


[0.6055835485458374, 0.7771602869033813]

In [12]:
model.evaluate(validation_s)

1593/1593 [==============================] - 49s 31ms/step - loss: 0.7757 - accuracy: 0.7240


[0.7757324576377869, 0.7239680290222168]

In [ ]:
valid = tf.keras.utils.image_dataset_from_directory(scaled_wo_rotate_validation, labels = 'inferred',
            image_size=(img_height,img_width), batch_size=batch_size, seed=42)
